In [0]:
%sql
USE CATALOG ecommerce;

In [0]:
# Explain query
spark.sql("""
          SELECT * 
          FROM silver.events
          WHERE event_type = 'purchase'
          """).explain(True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('event_type = purchase)
   +- 'UnresolvedRelation [silver, events], [], false

== Analyzed Logical Plan ==
event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string, ingestion_ts: timestamp, event_date: date, price_tier: string
Project [event_time#13280, event_type#13281, product_id#13282, category_id#13283L, category_code#13284, brand#13285, price#13286, user_id#13287, user_session#13288, ingestion_ts#13289, event_date#13290, price_tier#13291]
+- Filter (event_type#13281 = purchase)
   +- SubqueryAlias ecommerce.silver.events
      +- Relation ecommerce.silver.events[event_time#13280,event_type#13281,product_id#13282,category_id#13283L,category_code#13284,brand#13285,price#13286,user_id#13287,user_session#13288,ingestion_ts#13289,event_date#13290,price_tier#13291] parquet

== Optimized Logical Plan ==
Filter (isnotnull(event_

In [0]:
# Partitioned table
spark.sql("""
          CREATE TABLE IF NOT EXISTS silver.events_part
          USING DELTA
          PARTITIONED BY (event_date, event_type)
          AS
          SELECT * FROM silver.events
          """)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT * FROM silver.events_part
LIMIT 5;

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,ingestion_ts,event_date,price_tier
2019-11-07T07:00:55.000Z,cart,29501507,2053013566453186567,null,null,43.63,514232070,c32466d9-1039-4a97-88d9-d804a3933de8,2026-01-14T09:52:06.877Z,2019-11-07,mid
2019-11-07T07:32:13.000Z,cart,1002524,2053013555631882655,electronics.smartphone,apple,565.78,563392349,e26cfb3a-825e-4e80-b148-9b9abe6f41e0,2026-01-14T09:52:06.877Z,2019-11-07,premium
2019-11-07T09:21:41.000Z,cart,1005159,2053013555631882655,electronics.smartphone,xiaomi,211.07,554837336,820fdcff-037c-4687-8b63-9c7992b11d3d,2026-01-14T09:52:06.877Z,2019-11-07,premium
2019-11-07T10:35:14.000Z,cart,1004833,2053013555631882655,electronics.smartphone,samsung,170.05,550698799,ec71cfe8-3fc9-44e6-a95d-28d277cb5c88,2026-01-14T09:52:06.877Z,2019-11-07,premium
2019-11-07T11:29:23.000Z,cart,1005169,2053013555631882655,electronics.smartphone,samsung,209.8,568490743,21df68bc-d9e6-403f-b1df-21dcee37caf9,2026-01-14T09:52:06.877Z,2019-11-07,premium


In [0]:
# Optimize
spark.sql("""
          OPTIMIZE silver.events_part
          ZORDER BY (user_id, product_id)
          """).display()

path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 178, List(minCubeSize(107374182400), List(13, 639733708), List(178, 2952056699), 1, List(0, 0), 0, null), null, 0, 0, 191, 191, false, 0, 0, 1768729852634, 1768729855157, 8, 0, null, List(0, 0), null, 12, 12, 0, 0, null)"


In [0]:
# Benchmark
import time

start = time.time()

spark.sql("""
          SELECT * FROM silver.events_part
          WHERE user_id = 12345
          """).count()

print(f"Time Taken: {time.time() - start:.2f} seconds")

Time Taken: 0.58 seconds


In [0]:
# Cache for iterative queries (Serverless-safe)
cached = spark.sql("""
  SELECT *
  FROM silver.events_part
  WHERE event_type = 'purchase'
""")
print('Cached Count :', cached.count())

Cached Count : 1659690
